# Home Prices And Technical Jobs
We correlate recent home price data from Zillow for the largest 200 cities in the United States.  Once we have that, we run keyword searches against Dice and Indeed to get a total number of jobs published for various technology keywords my wife and I are interested in. (keywords = ['Business Intelligence', 'Java', '.NET', 'Data Analyst', 'SQL Server']). Our approach is pretty brute force since the apis are designed for single area searches, but within twenty minutes or less we have an aggregated result set (we save it as we go along periodically to data\output\JobAndHomeData.csv, and when we're done we have a df_final that you can play with and subset here (or just go grab the csv in Excel if you prefer).

Note you'll need an API publisher key (id) for Indeed before running this -- we set it in an environment variable "INDEED_PUBLISHER".  See https://www.indeed.com/publisher, then once your account is set up go to https://ads.indeed.com/jobroll/xmlfeed to see your publisher id.

In [9]:
# Standard python / pandas stuff
import pandas as pd
import numpy as np
import os

import webdata # This is our minimal API for querying Dice and Indeed data, in current directory.

df_price_per_sf = pd.read_csv('data/input/City_MedianSoldPricePerSqft_AllHomes.csv')
# Choose enough to end up with top 200 rows -- data is organized by rank already, so these are largest cities...
df_price_per_sf = df_price_per_sf.ix[:207, ["RegionID", "RegionName", "State", "Metro", "SizeRank", "2016-05"]]
df_price_per_sf = df_price_per_sf.dropna()
df_price_per_sf.index = df_price_per_sf["RegionID"]
df_price_per_sf = df_price_per_sf.drop("RegionID", axis=1)
df_price_per_sf = df_price_per_sf.rename(columns={"2016-05": "MedianPricePerSF"})


In [10]:
df_sold_price = pd.read_csv('data/input/City_MedianSoldPrice_AllHomes.csv')
# Choose enough to end up with top 200 rows -- data is organized by rank already, so these are largest cities...
df_sold_price = df_sold_price.ix[:207, ["RegionID", "2016-05"]]
df_sold_price = df_sold_price.dropna()
df_sold_price.index = df_sold_price["RegionID"]
df_sold_price = df_sold_price.drop("RegionID", axis=1)
df_sold_price = df_sold_price.rename(columns={"2016-05": "MedianSoldPrice"})


In [11]:
df_merged_prices = df_sold_price.merge(df_price_per_sf, left_index=True, right_index=True)
# Drop metro and re-arrange
df_merged_prices = df_merged_prices[['RegionName', 'State', 'SizeRank', 'MedianPricePerSF', 'MedianSoldPrice']]

df_final = df_merged_prices.copy()

# For testing purposes you can run just a subset of the data to make it go a lot faster.
# df_final = df_final[:3].copy()

cities = list(zip(list(df_final["RegionName"].values), list(df_final["State"].values)))


# Be patient
In our next step we'll run our keywords.
This will take several minutes to run, because each keyword lookup requires 200 api requests (one for each city).
For the 5 keywords below and 2 APIs -- that's 2000 round trips!  

This should take about 10 minutes or so to complete -- longer on a slow connection.

In [12]:
# Set up web runner
wd = webdata.WebData()
indeed_publisher_key = os.getenv("INDEED_PUBLISHER")
# The keywords to run
keywords = ['Business Intelligence', 'Java', '.NET', 'Data Analyst', 'SQL Server']
for keyword in keywords:
    print("Running keyword: {} on Dice".format(keyword))
    counts = wd.getDiceCountsForCities(cities, keyword)
    df_final[keyword + " (Dice)"] = counts
    print("Running keyword: {} on Indeed".format(keyword))
    counts = wd.getIndeedCountsForCities(indeed_publisher_key, cities, keyword)
    df_final[keyword + " (Indeed)"] = counts
    df_final.to_csv('data\output\JobAndHomeData.csv')
print("---------------- END OF RUN --------------------")


Running keyword: Business Intelligence on Dice
Running keyword: Business Intelligence on Indeed
Running keyword: Java on Dice
Running keyword: Java on Indeed
Running keyword: .NET on Dice
Running keyword: .NET on Indeed
Running keyword: Data Analyst on Dice
Running keyword: Data Analyst on Indeed
Running keyword: SQL Server on Dice
Running keyword: SQL Server on Indeed
---------------- END OF RUN --------------------


In [14]:
# Show the first 20 rows of data.
df_final[:20]

,RegionName,State,SizeRank,MedianPricePerSF,MedianSoldPrice,Business Intelligence (Dice),Business Intelligence (Indeed),Java (Dice),Java (Indeed),.NET (Dice),.NET (Indeed),Data Analyst (Dice),Data Analyst (Indeed),SQL Server (Dice),SQL Server (Indeed)
RegionID,,,,,,,,,,,,,,,
6181,New York,NY,1,509.9128,551600.0,233,234,1566,806,568,523,74,59,535,252
12447,Los Angeles,CA,2,411.2898,606800.0,108,74,479,219,312,260,16,25,239,97
17426,Chicago,IL,3,225.4799,297100.0,88,120,453,364,293,331,20,32,290,193
13271,Philadelphia,PA,4,125.8384,158800.0,89,53,395,188,226,230,32,22,234,102
40326,Phoenix,AZ,5,135.2148,212900.0,38,57,176,142,119,150,7,17,94,66
18959,Las Vegas,NV,6,120.1227,201350.0,10,9,21,21,15,35,0,3,17,16
54296,San Diego,CA,7,374.0997,523550.0,24,39,137,98,92,106,9,14,87,50
38128,Dallas,TX,8,152.2846,280038.1,83,123,485,286,283,260,24,33,293,119
33839,San Jose,CA,9,531.5886,798000.0,172,96,1266,629,201,145,49,32,236,56
